In [1]:
# Set up logging

import logging

logging.basicConfig(
    format="[%(asctime)s %(levelname)s %(threadName)s %(name)s:%(funcName)s:%(lineno)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S%z",
)
logging.getLogger("council").setLevel(logging.INFO)

In [2]:
# Council imports

from council.runners import Budget
from council.contexts import AgentContext, ChatHistory
from council.agents import Agent
from council.chains import Chain
from council.llm.openai_llm_configuration import OpenAILLMConfiguration
from council.llm.openai_llm import OpenAILLM

from skills import SectionWriterSkill, OutlineWriterSkill
from controller import WritingAssistantController
from evaluator import BasicEvaluatorWithSource

In [3]:
# Read environment variables and initialize OpenAI LLM instance

import dotenv
dotenv.load_dotenv()
openai_llm = OpenAILLM(config=OpenAILLMConfiguration.from_env())

In [4]:
# Create Skills

outline_skill = OutlineWriterSkill(openai_llm)
writing_skill = SectionWriterSkill(openai_llm)

In [5]:
# Create Chains

outline_chain = Chain(
    name="Outline Writer",
    description="Write or revise the outline (i.e. section headers) of a research article in markdown format. Only use this chain when the ARTICLE OUTLINE is missing or incomplete.",
    runners=[outline_skill]
)

writer_chain = Chain(
    name="Article Writer",
    description="Write or revise specific section bodies of a research article in markdown format. Use this chain to write the main research article content.",
    runners=[writing_skill]
)

In [6]:
# Create Controller

controller = WritingAssistantController(
    openai_llm,
    top_k_execution_plan=3
)

In [7]:
# Initialize Agent

chat_history = ChatHistory()
run_context = AgentContext(chat_history)
agent = Agent(controller, [outline_chain, writer_chain], BasicEvaluatorWithSource())

In [8]:
# Run Agent

chat_history.add_user_message("Write a detailed research article about the history of video games.")
result = agent.execute(run_context, Budget(1200))

[2023-07-25 19:00:20-0400 INFO MainThread council.agents.agent:execute:60] message="agent execution started"
[2023-07-25 19:00:20-0400 INFO MainThread council.agents.agent:execute:62] message="agent iteration started" iteration="1"
[2023-07-25 19:00:25-0400 INFO MainThread council.agents.agent:execute:71] message="chain execution started" chain="Outline Writer" execution_unit="Outline Writer: Create an outline for the research article about the history of video games. The outline should include sections such as Introduction, Early History, Evolution of Video Games, Impact on Society, and Conclusion."
[2023-07-25 19:00:25-0400 INFO chain_Outline Writer_0 council.skills.skill_base:execute_skill:82] message="skill execution started" skill="OutlineWriterSkill"
[2023-07-25 19:00:41-0400 INFO chain_Outline Writer_0 council.skills.skill_base:execute_skill:85] message="skill execution ended" skill="OutlineWriterSkill" skill_message="I've edited the outline and placed the response in the 'data'

In [9]:
print(result.messages[-1].message.message)

# The History of Video Games

## Introduction

The history of video games is a fascinating journey that spans over half a century, from the rudimentary games of the 1950s to the immersive and complex digital worlds of today. This article will delve into the evolution of video games, exploring the technological advancements, key milestones, and influential figures that have shaped this dynamic industry.

## The Origins of Video Games

The origins of video games can be traced back to the 1950s, when scientists began to experiment with computer games as a form of entertainment. The first video game, a simple simulation called "Tennis for Two," was created in 1958 by physicist William Higinbotham. This rudimentary game, played on an oscilloscope, laid the groundwork for the development of more complex games in the decades to come.

In the 1960s, video games took a significant leap forward. In 1962, a young MIT student named Steve Russell invented "Spacewar!", the first game intended for co

In [10]:
chat_history.add_user_message("Can you please add more details in each section?")
result = agent.execute(run_context, Budget(600))

[2023-07-25 20:08:05-0400 INFO MainThread council.agents.agent:execute:60] message="agent execution started"
[2023-07-25 20:08:05-0400 INFO MainThread council.agents.agent:execute:62] message="agent iteration started" iteration="3"
[2023-07-25 20:08:19-0400 INFO MainThread council.agents.agent:execute:71] message="chain execution started" chain="Article Writer" execution_unit="Article Writer: Expand the 'Early History of Video Games' section by adding more details about the first video games, origins and pioneers, the birth of arcade games, and the advent of home consoles."
[2023-07-25 20:08:19-0400 INFO chain_Article Writer_0 council.skills.skill_base:execute_skill:82] message="skill execution started" skill="SectionWriterSkill"
[2023-07-25 20:08:58-0400 INFO chain_Article Writer_0 council.skills.skill_base:execute_skill:85] message="skill execution ended" skill="SectionWriterSkill" skill_message="I've written or edited the article and placed it in the 'data' field."
[2023-07-25 20:08

In [11]:
print(result.messages[-1].message.message)

# The History of Video Games

## Introduction

The history of video games is a captivating journey that spans over half a century, from the rudimentary games of the 1950s to the immersive and complex digital worlds of today. This article will delve into the evolution of video games, exploring the technological advancements, key milestones, and influential figures that have shaped this dynamic industry.

## Early History of Video Games

### The First Video Games: Origins and Pioneers

The origins of video games can be traced back to the 1950s, when scientists began to experiment with computer games as a form of entertainment. The first video game, a simple simulation called "Tennis for Two," was created in 1958 by physicist William Higinbotham. This rudimentary game, played on an oscilloscope, laid the groundwork for the development of more complex games in the decades to come.

In the 1960s, video games took a significant leap forward. In 1962, a young MIT student named Steve Russell i